In [12]:
import pandas as pd
import numpy as np
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dateutil.relativedelta as relativedelta
import zipfile
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [53]:
zf = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/time series/rossmann sales/train.csv.zip')
train = pd.read_csv(zf.open('train.csv'))
store = pd.read_csv('/content/drive/My Drive/Colab Notebooks/time series/rossmann sales/store.csv')
df = pd.merge(train, store, how='inner', on='Store')
df = df.loc[df['Store'].isin([45, 175,224,337,1045])]
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
df

Exception ignored in: <function ZipFile.__del__ at 0x7ff96fe3b490>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
  File "/usr/lib/python3.10/zipfile.py", line 1843, in close
  File "/usr/lib/python3.10/zipfile.py", line 1943, in _fpclose
OSError: [Errno 107] Transport endpoint is not connected
<ipython-input-53-88dfb85425ad>:2: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
40344,45,5,2015-07-31,6301,442,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
40345,45,4,2015-07-30,6063,445,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
40346,45,3,2015-07-29,5341,387,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
40347,45,2,2015-07-28,5504,365,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
40348,45,1,2015-07-27,7450,500,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953288,1045,6,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN
953289,1045,5,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN
953290,1045,4,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN
953291,1045,3,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN


In [15]:
pd.DataFrame(data={'Dtypes': df.dtypes, 'Isnull': df.isnull().sum(), 'Nunique': df.nunique()}, index=df.columns)

,Dtypes,Isnull,Nunique
Store,int64,0,5
DayOfWeek,int64,0,7
Date,datetime64[ns],0,942
Sales,int64,0,2918
Customers,int64,0,918
Open,int64,0,2
Promo,int64,0,2
StateHoliday,object,0,5
SchoolHoliday,int64,0,2
StoreType,object,0,3


In [8]:
df.loc[df['CompetitionOpenSinceMonth'].isnull(), ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']]

,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear
157652,4130.0,NaN,NaN
157653,4130.0,NaN,NaN
157654,4130.0,NaN,NaN
157655,4130.0,NaN,NaN
157656,4130.0,NaN,NaN
...,...,...,...
203091,7930.0,NaN,NaN
203092,7930.0,NaN,NaN
203093,7930.0,NaN,NaN
203094,7930.0,NaN,NaN


In [54]:
import datetime
def compet_process(df):
    df['Competition'] = np.nan
    for s in [45,175,224,337,1045]:
        cosy, cosm = store.loc[store['Store'] == s, ['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth']].values[0]
        if pd.isnull(cosm):
            df.loc[df['Store'] == s, 'Competition'] = 1
        else:
            sd = datetime.datetime(cosy.astype(int), cosm.astype(int), 1)
            df.loc[(df['Store'] == s) & (df['Date'] >= sd), 'Competition'] = 1
            df.loc[(df['Store'] == s) & (df['Date'] < sd), 'Competition'] = 0
    return df

df = compet_process(df)
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Competition
40344,45,5,2015-07-31,6301,442,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN,1.0
40345,45,4,2015-07-30,6063,445,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN,1.0
40346,45,3,2015-07-29,5341,387,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN,1.0
40347,45,2,2015-07-28,5504,365,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN,1.0
40348,45,1,2015-07-27,7450,500,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953288,1045,6,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN,0.0
953289,1045,5,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN,0.0
953290,1045,4,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN,0.0
953291,1045,3,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,NaN,0.0


In [55]:
def promo2_process(df):
    for s in [45,175,224,337,1045]:
        p2 = store.loc[store['Store'] == s, 'Promo2'].values
        if p2 == 1:
            y, w = store.loc[store['Store'] == s, ['Promo2SinceYear', 'Promo2SinceWeek']].values[0]
            sd = datetime.date(y.astype(int),1,1)+relativedelta.relativedelta(weeks=+int(w.astype(int)))
            df.loc[(df['Store'] == s) & (df['Date'].dt.date > sd), 'Promo2'] = 1
            df.loc[(df['Store'] == s) & (df['Date'].dt.date < sd), 'Promo2'] = 0
        else:
            df.loc[df['Store'] == s, 'PromoInterval'] = 'No promo2'
    return df

df = promo2_process(df)
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Competition
40344,45,5,2015-07-31,6301,442,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40345,45,4,2015-07-30,6063,445,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40346,45,3,2015-07-29,5341,387,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40347,45,2,2015-07-28,5504,365,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40348,45,1,2015-07-27,7450,500,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953288,1045,6,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953289,1045,5,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953290,1045,4,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953291,1045,3,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0


In [19]:
df['PromoInterval'].unique()[1].split(',')

['Jan', 'Apr', 'Jul', 'Oct']

In [36]:
df.loc[df['Store']==337, 'PromoInterval'].unique()[0].split(',')

['Feb', 'May', 'Aug', 'Nov']

In [28]:
df.loc[df['Store']==45, 'Date'].dt.strftime('%b').isin(['Jan', 'Apr', 'Jul', 'Oct'])

40344    True
40345    True
40346    True
40347    True
40348    True
         ... 
41281    True
41282    True
41283    True
41284    True
41285    True
Name: Date, Length: 942, dtype: bool

In [56]:
def promo2interval_process(df):
    for s in [45,175,224,337,1045]:
        interval = df.loc[df['Store']==s, 'PromoInterval'].unique()[0].split(',') #ex: ['Feb', 'May', 'Aug', 'Nov']
        if interval=='No promo2':
            df.loc[df['Store'] == s, 'Promo2'] = 0
        else:
            mask = df.loc[df['Store']==s, 'Date'].dt.strftime('%b').isin(interval) #get month from date and check if it is in the interval
            mask = mask*df.loc[df['Store'] == s, 'Promo2'].values
            df.loc[df['Store'] == s, 'Promo2'] = mask
    return df

df = promo2interval_process(df)
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Competition
40344,45,5,2015-07-31,6301,442,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40345,45,4,2015-07-30,6063,445,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40346,45,3,2015-07-29,5341,387,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40347,45,2,2015-07-28,5504,365,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
40348,45,1,2015-07-27,7450,500,1,1,0,1,d,a,9710.0,2.0,2014.0,0,NaN,NaN,No promo2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953288,1045,6,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953289,1045,5,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953290,1045,4,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0
953291,1045,3,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,12.0,2013.0,0,NaN,NaN,No promo2,0.0


In [52]:
False*True

0

In [57]:
df.loc[(df['Store']==337)&(df['Promo2']==True), ['Date', 'Promo2', 'Promo2SinceYear','PromoInterval']]

,Date,Promo2,Promo2SinceYear,PromoInterval
306085,2015-05-31,1,2014.0,"Feb,May,Aug,Nov"
306086,2015-05-30,1,2014.0,"Feb,May,Aug,Nov"
306087,2015-05-29,1,2014.0,"Feb,May,Aug,Nov"
306088,2015-05-28,1,2014.0,"Feb,May,Aug,Nov"
306089,2015-05-27,1,2014.0,"Feb,May,Aug,Nov"
...,...,...,...,...
306281,2014-11-16,1,2014.0,"Feb,May,Aug,Nov"
306282,2014-11-15,1,2014.0,"Feb,May,Aug,Nov"
306283,2014-11-14,1,2014.0,"Feb,May,Aug,Nov"
306284,2014-11-13,1,2014.0,"Feb,May,Aug,Nov"


In [58]:
df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'Competition'],
      dtype='object')

In [59]:
df.drop(columns=['DayOfWeek','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], inplace=True)
df.to_csv('/content/drive/My Drive/Colab Notebooks/time series/rossmann sales/sub_rossmann.csv', index=False)